<a href="https://colab.research.google.com/github/juanmaniglia/Guia_Tensorflow_Keras/blob/main/Clase_07/clase_07_2_train_gan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Modulo 7 : GANs**
* Instructor: [Juan Maniglia](https://juanmaniglia.github.io)

# Google CoLab Instructions

In [ ]:
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    COLAB = True
    print("Note: using Google CoLab")
    %tensorflow_version 2.x
except:
    print("Note: not using Google CoLab")
    COLAB = False

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
Note: using Google CoLab
TensorFlow 2.x selected.


# Modulo 7.2: Entrenar StyleGAN2 con tus imagenes

Entrenar GAN con StyleGAN requiere muchos recursos. Para las GAN faciales de alta resolución entrenadas por NVIDIA, los investigadores usaron computadoras con 8 GPU de gama alta. La GPU utilizada por NVIDIA es una A100, que tiene más memoria y núcleos que la P100 o la V100 que ofrece incluso Colab Pro+. En esta parte, usaremos StyleGAN2 para entrenar, en lugar de StyleGAN3. Puede usar redes entrenadas con StyleGAN2 de StyleGAN3; sin embargo, no se entrenarán tan bien como StyleGAN3.

Desafortunadamente, StyleGAN3 requiere un uso intensivo de cómputo y funcionará lentamente en cualquier GPU que no tenga la última tecnología de Ampere. Debido a que Colab no proporciona dicha tecnología, la guía de capacitación utilizara el nivel StyleGAN2.

Asegúrese de estar ejecutando este portátil con un tiempo de ejecución de GPU. Los GAN se pueden entrenar con Google Colab Free o Pro. Recomiendo al menos la versión Pro debido a mejores instancias de GPU, tiempos de ejecución más largos y tiempos de espera.


Sus datos de entrenamiento y redes neuronales entrenadas se almacenarán en GDRIVE. Para GAN, diseño mi GDRIVE de esta manera:

* ./data/gan/images: imágenes RAW en las que deseo entrenar.
* ./data/gan/datasets: conjuntos de datos de entrenamiento reales que convierto a partir de las imágenes sin procesar.
* ./data/gan/experiments: la salida de StyleGAN2, mis vistas previas de imágenes y las instantáneas de red guardadas.

La unidad se monta en la siguiente ubicación.

```
/content/drive/MyDrive/data
```

### ¿Qué tipo de GPU tienes?

Por lo general el CoLab gratuito nos otorga una GPU T4.

* 1024x1024 - T4 - 2188 sec/tick (CoLab Free)

By comparison, a 1024x1024 GAN trained with StyleGAN3 on a V100 is 3087 sec/tick.

CoLab gratuito desconectará un script en ejecución perfectamente bueno si no interactúa durante unas horas. A continuación se describe cómo solventar este problema.

* [¿Cómo evitar que Google Colab se desconecte?](https://stackoverflow.com/questions/57113226/how-to-prevent-google-colab-from-disconnecting)


In [1]:
!pip install torch==1.8.1 torchvision==0.9.1
!git clone https://github.com/NVlabs/stylegan2-ada-pytorch.git
!pip install ninja

ERROR: Could not find a version that satisfies the requirement torch==1.8.1 (from versions: 1.9.0, 1.10.0, 1.10.1)
ERROR: No matching distribution found for torch==1.8.1
Cloning into 'stylegan2-ada-pytorch'...
remote: Enumerating objects: 128, done.
remote: Total 128 (delta 0), reused 0 (delta 0), pack-reused 128
Receiving objects: 100% (128/128), 1.12 MiB | 2.33 MiB/s, done.
Resolving deltas: 100% (58/58), done.


### Encuentra tus archivos

La unidad se monta en la siguiente ubicación.

```
/content/drive/MyDrive/data
```

Puede ser útil usar un comando ```ls``` para establecer la ruta exacta de sus imágenes.

In [ ]:
!ls /content/drive/MyDrive/data/gan/images

### Convierte tus imágenes

In [ ]:
!python /content/stylegan2-ada-pytorch/dataset_tool.py --source /content/drive/MyDrive/data/gan/images/circuit --dest /content/drive/MyDrive/data/gan/dataset/circuit

El siguiente comando se puede utilizar para borrar el conjunto de datos recién creado. Si algo sale mal y necesita limpiar sus imágenes y volver a ejecutar el comando anterior, debe eliminar su directorio de conjunto de datos creado parcialmente.

In [ ]:
#!rm -R /content/drive/MyDrive/data/gan/dataset/circuit/*

In [ ]:
import torch, dnnlib
def expand_seed(seeds, vector_size):
  result = []

  for seed in seeds:
    rnd = np.random.RandomState(seed)
    result.append( rnd.randn(1, vector_size) ) 
  return result

URL = "https://github.com/jeffheaton/pretrained-gan-fish/releases/"\
 "download/1.0.0/fish-gan-2020-12-09.pkl"

print(f'Loading networks from "{URL}"...')
device = torch.device('cuda')
with dnnlib.util.open_url(URL) as f:
    G = legacy.load_network_pkl(f)['G_ema'].to(device) # type: ignore

vector_size = G.z_dim
# range(8192,8300)
seeds = expand_seed( [8192+1,8192+9], vector_size)
#generate_images(Gs, seeds,truncation_psi=0.5)
print(seeds[0].shape)

# Limpia tus imágenes

Es importante que todas las imágenes tengan las mismas dimensiones y profundidad de color. Este código puede identificar imágenes que tienen problemas.

In [ ]:
from os import listdir
from os.path import isfile, join
import os
from PIL import Image
from tqdm.notebook import tqdm

IMAGE_PATH = '/content/drive/MyDrive/data/gan/images/fish'
files = [f for f in listdir(IMAGE_PATH) if isfile(join(IMAGE_PATH, f))]

base_size = None
for file in tqdm(files):
  file2 = os.path.join(IMAGE_PATH,file)
  img = Image.open(file2)
  sz = img.size
  if base_size and sz!=base_size:
    print(f"Inconsistant size: {file2}")
  elif img.mode!='RGB':
    print(f"Inconsistant color format: {file2}")
  else:
    base_size = sz


### Realizar entrenamiento inicial

Este código realiza el entrenamiento inicial. Configure SNAP lo suficientemente bajo como para obtener una instantánea antes de que Colab lo obligue a salir.

In [ ]:
import os

# Modifique estas para satisfacer sus necesidades.
EXPERIMENTS = "/content/drive/MyDrive/data/gan/experiments"
DATA = "/content/drive/MyDrive/data/gan/dataset/circuit"
SNAP = 10

# Cree el comando y ejecútelo
cmd = f"/usr/bin/python3 /content/stylegan2-ada-pytorch/train.py --snap {SNAP} --outdir {EXPERIMENTS} --data {DATA}"
!{cmd}

# Reanudar el entrenamiento

Ahora puede reanudar el entrenamiento después de que algo lo interrumpa en el paso anterior.

In [ ]:
import os

# Modifique estas para satisfacer sus necesidades.
EXPERIMENTS = "/content/drive/MyDrive/data/gan/experiments"
NETWORK = "network-snapshot-000100.pkl"
RESUME = os.path.join(EXPERIMENTS, "00008-circuit-auto1-resumecustom", NETWORK)
DATA = "/content/drive/MyDrive/data/gan/dataset/circuit"
SNAP = 10

# Cree el comando y ejecútelo
cmd = f"/usr/bin/python3 /content/stylegan2-ada-pytorch/train.py --snap {SNAP} --resume {RESUME} --outdir {EXPERIMENTS} --data {DATA}"
!{cmd}